<h1 style="border: thick double #32a1ce; text-align:center;border-radius:35px">
Modélisation statistiques
</h1>

In [ ]:
# Return a new path pointing to the current working directory
HOME_DIR = Path.cwd().parent

# create a variable for data directory
DATA_DIR = Path(HOME_DIR, "machine learning")

print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

In [ ]:
data_cleaned = pd.read_csv(Path(DATA_DIR, "data_cleaned.csv"))

<h1 style="border: thick double #32a1ce; text-align:center;border-radius:35px">
I- feature scaling des variables
</h1>

Les variables de notre jeu de données ayant une très grande échelle, nous allons faire du feature scaling (les remettre à l'echelle) pour rendre nos données homogènes. La méthode privilégiée dans notre cas est la standardisation car les méthodes (regression) utilisées reposent sur l'hypothèse de normalité.

In [ ]:
data_cleaned.columns

In [ ]:
col_num = data_cleaned.select_dtypes(['int64','float64']).columns
col_num

In [ ]:
Var_cat=data_cleaned.select_dtypes(['object']).columns
Var_cat

In [ ]:
var_norm=[ 'NumberofFloors', 'PropertyGFATotal',
       'SourceEUI(kBtu/sf)', 'SteamUse(kBtu)',
       'Electricity(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions',
       'GHGEmissionsIntensity', 'NumberOfUse', 'BuildingAge',
       'SiteEnergyUse(log(kBtu))']
var_cat=['BuildingType', 'PrimaryPropertyType', 'Neighborhood']

In [ ]:
data_cleaned[var_norm].isnull().sum()

In [ ]:
data_cleaned.info()

In [ ]:
data_mod=data_cleaned[['NumberofFloors', 'PropertyGFATotal',
       'SourceEUI(kBtu/sf)', 'SteamUse(kBtu)',
       'Electricity(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions',
       'GHGEmissionsIntensity', 'NumberOfUse', 'BuildingAge',
       'SiteEnergyUse(log(kBtu))','BuildingType', 'PrimaryPropertyType', 'Neighborhood']]

<h2 style= "text-align:left">
I.1- Standardisation  des variables numériques 
</h2>

<h3 style= "text-align:left">
I.1.a-  Standarscaler 
</h3>

In [ ]:
# standardisation des variables numériques
data_mod[var_norm] = StandardScaler().fit_transform(data_mod[var_norm])

Verifiaction de la normalisation

In [ ]:
def verification_normalisation(data_mod):
      for variable_norm in data_mod :
        data_mod_col=data_mod[variable_norm]
        moyenne=np.mean(data_mod_col)
        ecartType = np.std(data_mod_col)

        print("les statistiques pour la variable {}" .format(variable_norm))
        print("La moyenne est de : {} ".format(round(abs(moyenne), 2)))
        print("L'écart type est de : {} ".format(round(abs(ecartType), 2)))
        print(" ")
        print(" ")

In [ ]:
verification_normalisation(data_cleaned[var_norm])

In [ ]:
<h1 style= "text-align:left">
Recherche du meilleur algorithme 
</h1>